# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [ ]:
import findspark
findspark.init('/home/quanttraderkim/spark-2.3.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('tree').getOrCreate()

In [6]:
df = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [7]:
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [14]:
df.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [11]:
from pyspark.ml.classification import RandomForestClassifier

In [13]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer

In [16]:
from pyspark.ml import Pipeline

In [15]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [44]:
final_df = assembler.transform(df).select('features', 'Spoiled')

In [45]:
rf = RandomForestClassifier(labelCol='Spoiled', numTrees=100) # Set Default First

In [46]:
model = rf.fit(final_df)

In [47]:
feat_im = model.featureImportances
print('Pres_A: {}'.format(feat_im[0]))
print('Pres_B: {}'.format(feat_im[1]))
print('Pres_C: {}'.format(feat_im[2]))
print('Pres_D: {}'.format(feat_im[3]))


Pres_A: 0.020498723378992854
Pres_B: 0.021206122925107793
Pres_C: 0.9364779511066231
Pres_D: 0.02181720258927626


Pres_C is an overriding feature!

In [33]:
train_df, test_df = final_df.randomSplit([0.7,0.3])

In [34]:
train_model = rf.fit(train_df)

In [35]:
pred = train_model.transform(test_df)

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
evaluator = MulticlassClassificationEvaluator(labelCol='Spoiled', metricName='accuracy')

In [40]:
evaluator.evaluate(pred)

0.9930255607720397